In [34]:
import pandas as pd 
import numpy  as np

data1 = {'trip_id'      : [1,2,3,4,5,6,7],
         'rider_id'     : [1,2,3,4,1,2,3],
         'driver_id'    : [10,11,12,10,11,12,11],
         'status'       : ['completed','cancelled_by_driver',
                           'completed','cancelled_by_rider',
                           'completed','completed','completed'],
         'request_date' : ['01-OCT-20',
                          '01-OCT-20',
                          '01-OCT-20',
                          '02-OCT-20',
                          '02-OCT-20',
                          '02-OCT-20',
                          '03-OCT-20']
        }

data2 = {'user_id' : [1,2,3,4,10,11,12],
         'banned'  : ['no','yes','no','no','no','no','no'],
         'type'    : ['rider','rider','rider','drider','driver','driver','driver']
        }
                    

trips = pd.DataFrame(data1)
users = pd.DataFrame(data2)

In [35]:
trips['request_date'] = pd.to_datetime(trips['request_date'])

In [36]:
trips

,trip_id,rider_id,driver_id,status,request_date
0,1,1,10,completed,2020-10-01
1,2,2,11,cancelled_by_driver,2020-10-01
2,3,3,12,completed,2020-10-01
3,4,4,10,cancelled_by_rider,2020-10-02
4,5,1,11,completed,2020-10-02
5,6,2,12,completed,2020-10-02
6,7,3,11,completed,2020-10-03


In [37]:
users

,user_id,banned,type
0,1,no,rider
1,2,yes,rider
2,3,no,rider
3,4,no,drider
4,10,no,driver
5,11,no,driver
6,12,no,driver


In [38]:
riders  = users[['user_id','banned']].copy()
drivers = users[['user_id','banned']].copy()
riders.rename(columns  = {'user_id':'rider_id', 'banned':'isRiderBanned'},  inplace = True)
drivers.rename(columns = {'user_id':'driver_id','banned':'isDriverBanned'}, inplace = True)

In [39]:
trips = pd.merge(trips, riders, on = 'rider_id', how = 'left')

In [40]:
trips

,trip_id,rider_id,driver_id,status,request_date,isRiderBanned
0,1,1,10,completed,2020-10-01,no
1,2,2,11,cancelled_by_driver,2020-10-01,yes
2,3,3,12,completed,2020-10-01,no
3,4,4,10,cancelled_by_rider,2020-10-02,no
4,5,1,11,completed,2020-10-02,no
5,6,2,12,completed,2020-10-02,yes
6,7,3,11,completed,2020-10-03,no


In [41]:
trips = pd.merge(trips, drivers, on = 'driver_id', how = 'left')

In [42]:
trips

,trip_id,rider_id,driver_id,status,request_date,isRiderBanned,isDriverBanned
0,1,1,10,completed,2020-10-01,no,no
1,2,2,11,cancelled_by_driver,2020-10-01,yes,no
2,3,3,12,completed,2020-10-01,no,no
3,4,4,10,cancelled_by_rider,2020-10-02,no,no
4,5,1,11,completed,2020-10-02,no,no
5,6,2,12,completed,2020-10-02,yes,no
6,7,3,11,completed,2020-10-03,no,no


In [43]:
f = np.where((trips['isRiderBanned'] != 'yes') & (trips['isDriverBanned'] != 'yes') & (trips['request_date'].dt.day <= 2))
df = trips.loc[f]

In [44]:
df

,trip_id,rider_id,driver_id,status,request_date,isRiderBanned,isDriverBanned
0,1,1,10,completed,2020-10-01,no,no
2,3,3,12,completed,2020-10-01,no,no
3,4,4,10,cancelled_by_rider,2020-10-02,no,no
4,5,1,11,completed,2020-10-02,no,no


In [45]:
df['cancellation_rate'] = 1 - np.where(df['status'] == 'completed', 1, 0)

In [46]:
df

,trip_id,rider_id,driver_id,status,request_date,isRiderBanned,isDriverBanned,cancellation_rate
0,1,1,10,completed,2020-10-01,no,no,0
2,3,3,12,completed,2020-10-01,no,no,0
3,4,4,10,cancelled_by_rider,2020-10-02,no,no,1
4,5,1,11,completed,2020-10-02,no,no,0


In [49]:
df = pd.DataFrame(df.groupby('request_date')['cancellation_rate'].mean())

In [50]:
df

,cancellation_rate
request_date,
2020-10-01,0.0
2020-10-02,0.5
